In [ ]:
import json
import pandas as pd
#!pip install openai -U
from openai import OpenAI
from enum import Enum
from typing import Optional, List, Tuple
from pydantic import BaseModel
from pydantic import Field

In [ ]:
with open('../key/key.json') as f:
    k = json.load(f)['key']

In [ ]:
# create a list of 50 human emotions
def get_emotions_list():
    client = OpenAI(api_key=k)
    class Chr_answer(BaseModel):
        Characteristics: List[str] = Field(None, description="Emotions as a list of strings")

    #few shot prompt with api
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "Be a helpful assistant."},
            {"role": "system", "content": "Find 50 non-redundant human emotions. These emotions should be different from one another, meaning that for example, of the emotions 'joy' and 'happiness', only pick one. Of 'Shame' and 'Embarrassment' and of 'Envy' and 'Jealousy', only pick one. So on and so forth."},
            {"role": "system", "content": "Stop finding if you've already found 50."},
            {"role": "system", "content": "DONT MAKE ANY MISTAKES, check if you did any."},
            {"role": "user", "content": "Give me about 50 emotions addressing non-redundant and different human feelings."}
        ],
        response_format=Chr_answer,
    )

    #you can also proceed from parsed eg json.loads(completion_song.choices[0].message.parsed.json())
    out = json.loads(completion.choices[0].message.content)
    out = list(out.values())[0]
    return out

In [ ]:
emotions = get_emotions_list()
Characteristic = Enum('Characteristic', dict([(emotion, emotion) for emotion in emotions]))

In [ ]:


class EmotionalAssociationScore(BaseModel):
    emotion: Characteristic
    score: float

class EmotionalAssociationScores(BaseModel):
    associations: List[EmotionalAssociationScore] = Field(description="A list of emotions and associated scores")

def emotional_association_scores(
        thing, 
        emotions = emotions,
        min_value = 0,
        max_value = 500
    ):
    prompt = f"""Assign emotional association scores between {min_value} and {max_value} for the provided thing.
            Assign a score for each of the following emotions.
            Ensure the scores reflect the association strength for the specified thing.
            Thing:
            {thing}
            """
            
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "Be a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        response_format=EmotionalAssociationScores,
    )
    #output returns in the defined pydantic style
    output = completion.choices[0].message.parsed
    return output.json()

In [ ]:
thing = brands[3]
completion = emotional_association_scores(thing)
completion

In [ ]:
#tried nested prompt but decided to go with one prompt and a list comprehension
[(brand, emotional_association_scores(thing)) for brand in brands[:3]]

In [ ]:
#List 100 best selling American clothing brands 
client = OpenAI(api_key=k)
class Chr_answer(BaseModel):
    Characteristics: List[str] = Field(None, description="Brands as a list of strings")

#few shot prompt with api
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "system", "content": "Find 10 non-redundant best selling American clothing brands."},
        {"role": "system", "content": "DONT MAKE ANY MISTAKES, check if you did any."},
        {"role": "user", "content": "Give me 10 best selling American clothing brands."}
    ],
    response_format=Chr_answer,
)

#you can also proceed from parsed eg json.loads(completion_song.choices[0].message.parsed.json())
out = json.loads(completion.choices[0].message.content)
out = list(out.values())[0]
brands = out
print(brands)
print(len(brands))

In [ ]:
import openai
from pydantic import BaseModel, Field
from typing import List, Tuple

# Define the Pydantic model
class RelationResponse(BaseModel):
    pair: Tuple[str, str]  # First str is object1, second str is object2 linked to the association score
    association_score: float  # Association scores for each object2 in relation to object1

class RelationResponses(BaseModel):
    objects: List[RelationResponse] = Field(None, description="List of object emotion pairs and association scores")

# # Define a function to convert the model instance to the desired output structure
# def format_output(model_instance: PairAssociation) -> dict:
#     # Convert Enum to string
#     pair_str = (model_instance.pair[0], model_instance.pair[1].value)
#     # Create the final structure
#     return {'pair': pair_str, 'association_value': model_instance.association_value}

# class RelationResponse(BaseModel):
#     object1: str
#     associations: Dict[str, float]  # Association scores for each object2 in relation to object1

class OpenAIRelationQuantifier:
    @staticmethod
    def _generate_prompt(object1: str, list2: List[str], min_value: float, max_value: float) -> str:
        """Generate the prompt for GPT to evaluate the association between one object and a list of other objects. 
        Minimum and maximum value of the normalized range."""
        object2_str = ', '.join(list2)
        return f"For the following object '{object1}', assign a score between {min_value} and {max_value} to each object in this list: [{object2_str}]. " \
               f"Please return the result as 'Object1 linked to the association score, object2 linked to the association score' pairs, and ensure the scores reflect the association strength."

    @staticmethod
    def quantify_relations(list1: List[str], list2: List[str], min_value: float, max_value: float, model: str, api_key: str) -> List[RelationResponse]:
        """Quantify the relationships between each object1 and all objects in list2 using GPT."""
        client = OpenAI(api_key=api_key)
        results = []
        for object1 in list1:
            prompt = OpenAIRelationQuantifier._generate_prompt(object1, list2, min_value, max_value)
            print(object1, prompt)
            # Send the prompt to GPT using beta.chat.completions.parse method
            response = client.beta.chat.completions.parse(
                model=model,
                messages=[
                    {"role": "system", "content": "You are an expert at analyzing relationships between concepts."},
                    {"role": "user", "content": prompt}
                ],
                response_format=RelationResponse
            )

            # Extract GPT's parsed response
            gpt_reply = response.choices[0].message.content
            print(gpt_reply)
            # score_line = gpt_reply.splitlines()[0]
            
            # score = float(score_line.split()[-1].rstrip('.'))  # Assumes the score is at the end of the first line

            # # Append the result
            # results.append(RelationResponse(pair=pair, association_score=score))

        return gpt_reply

# Example Usage

# Quantify the relationships between objects in the two lists
relations = OpenAIRelationQuantifier.quantify_relations(
    list1=brands[:3], 
    list2=emotions[:3], 
    min_value=0, 
    max_value=9, 
    model="gpt-4o-2024-08-06", 
    api_key=k
    )

relations

In [ ]:
# #as dict
# from pydantic import BaseModel, Field
# from typing import Tuple
# class RelationRequest(BaseModel):
#     list1: List[str]  # First list of objects (object1)
#     list2: List[str]  # Second list of objects (object2)
#     model: str = Field(default="gpt-4o-2024-08-06")  # Updated OpenAI model version
#     min_value: float = 0  # Minimum value of the normalized range
#     max_value: float = 500  # Maximum value of the normalized range (set to 500)

# class ObjectPair(BaseModel):
#     object1: str
#     obj2: str

# class RelationResponse(BaseModel):
#     pair: ObjectPair
#     association_score: float  # The score provided by GPT


# class OpenAIRelationQuantifier:
#     @staticmethod
#     def _generate_prompt(object1: str, object2_list: List[str], min_value: float, max_value: float) -> str:
#         """Generate the prompt for GPT to evaluate the association between one object and a list of other objects."""
#         prompt = f"For the object '{object1}', please provide a score between {min_value} and {max_value} for its association with each of the following objects:"
        
#         for idx, obj2 in enumerate(object2_list, 1):
#             prompt += f"\n{idx}. {obj2}"
#             print(obj2)

#         prompt += "\n\nReturn the result as 'object1, obj2' pairs and their associated scores"

#         return prompt

#     @staticmethod
#     def quantify_relations(request: RelationRequest, api_key: str) -> List[RelationResponse]:
#         """Quantify the relationships between each object1 and all objects in object2 using GPT."""
#         openai.api_key = api_key  # Set the API key for OpenAI

#         results = []
#         for object1 in request.list1:
#             prompt = OpenAIRelationQuantifier._generate_prompt(object1, request.list2, request.min_value, request.max_value)

#             # Send the prompt to GPT
#             response = client.beta.chat.completions.parse(
#                 model='gpt-4o-2024-08-06',
#                 messages=[
#                     {"role": "system", "content": "You are an expert at analyzing relationships between concepts."},
#                     {"role": "user", "content": prompt}
#                 ],
#                 response_format=RelationResponse
#             )

#             # Extract GPT's parsed response
#             results.append(response.choices[0].message.content)
#         return results


# api_key = k

# # Define two lists of objects to compare
# list1 = brands[:3]
# list2 = emotions[:2]

# # Create the relation request object
# relation_request = RelationRequest(list1=list1, list2=list2, min_value=0, max_value=500)

# # Quantify the relationships between objects in the two lists
# relations = OpenAIRelationQuantifier.quantify_relations(relation_request, api_key=api_key)

# relations

In [ ]:
# #as joined str in a list
# class RelationRequest(BaseModel):
#     list1: List[str]  # First list of objects (object1)
#     list2: List[str]  # Second list of objects (object2)
#     model: str = Field(default="gpt-4o-2024-08-06")  # Updated OpenAI model version
#     min_value: float = 0  # Minimum value of the normalized range
#     max_value: float = 500  # Maximum value of the normalized range (set to 500)

# class ObjectPair(BaseModel):
#     object1: str
#     obj2: str

# class RelationResponse(BaseModel):
#     pair: ObjectPair
#     association_score: float  # The score provided by GPT

# class OpenAIRelationQuantifier:
#     @staticmethod
#     def _generate_prompt(object1: str, object2_list: List[str], min_value: float, max_value: float) -> str:
#         """Generate the prompt for GPT to evaluate the association between one object and a list of other objects."""
#         object2_str = ', '.join(object2_list)
#         return f"For the following object '{object1}', assign a score between {min_value} and {max_value} to each object in this list: [{object2_str}]. " \
#                f"Please return the result as 'object1 obj2' pairs, and ensure the scores reflect the association strength."

#     @staticmethod
#     def quantify_relations(request: RelationRequest, api_key: str) -> List[RelationResponse]:
#         """Quantify the relationships between each object1 and all objects in object2 using GPT."""
#         openai.api_key = api_key  # Set the API key for OpenAI

#         results = []
#         for object1 in request.list1:
#             prompt = OpenAIRelationQuantifier._generate_prompt(object1, request.list2, request.min_value, request.max_value)

#             response = client.beta.chat.completions.parse(
#                 model=request.model,
#                 messages=[
#                     {"role": "system", "content": "You are an expert at analyzing relationships between concepts."},
#                     {"role": "user", "content": prompt}
#                 ],
#                 response_format=RelationResponse
#             )
            
#             results.append(relations.choices[0].message.content)

#         return results


# api_key = k

# # Define two lists of objects to compare
# list1 = ["Nike", "lululemon"]
# list2 = ["Joy","liberated"]

# # Create the relation request object
# relation_request = RelationRequest(list1=list1, list2=list2, min_value=0, max_value=500)

# # Quantify the relationships between objects in the two lists
# relations = OpenAIRelationQuantifier.quantify_relations(relation_request, api_key=api_key)
# relations


In [ ]:
# a method
#embedding dimension is emotions
#talk about options
#get the brands, go through 50 emotins at a time
#cosine: normalize first: l2 norm = 1


In [ ]:
# #Goal: Get binary association of 100 brands and 1 book with 50 emotions. Multibinarize and do a correlation. in the example, no correlation with any of the brands
# # create a list of 50 human emotions
# client = OpenAI(api_key=k)
# class Chr_answer(BaseModel):
#     Characteristics: List[str] = Field(None, description="Emotions as a list of strings")

# #few shot prompt with api
# completion = client.beta.chat.completions.parse(
#     model="gpt-4o-2024-08-06",
#     messages=[
#         {"role": "system", "content": "Be a helpful assistant."},
#         {"role": "system", "content": "Find 50 non-redundant human emotions. These emotions should be different from one another, meaning that for example, of the emotions 'joy' and 'happiness', only pick one. Of 'Shame' and 'Embarrassment' and of 'Envy' and 'Jealousy', only pick one. So on and so forth."},
#         {"role": "system", "content": "Stop finding if you've already found 50."},
#         {"role": "system", "content": "DONT MAKE ANY MISTAKES, check if you did any."},
#         {"role": "user", "content": "Give me about 50 emotions addressing non-redundant and different human feelings."}
#     ],
#     response_format=Chr_answer,
# )

# #you can also proceed from parsed eg json.loads(completion_song.choices[0].message.parsed.json())
# out = json.loads(completion.choices[0].message.content)
# out = list(out.values())[0]
# emotions = out
# print(emotions)
# print(len(emotions))

# #List 100 best selling American clothing brands and, for each brand, list which one of the emotions in the given list are most closely associated with (Either associated or not, binary)."
# client = OpenAI(api_key=k)

# Characteristic = Enum('Characteristic', dict([(i, i) for i in emotions])) #MyEnumType = Enum('MyEnumType', myEnumStrings)

# class BrandAttributes(BaseModel):
#     brand_name: str = Field(None, description=" name as str")
#     characteristics: List[Characteristic] = Field(None, description=" list of characteristics as str")

# class BrandChars(BaseModel):
#     a: List[BrandAttributes] = Field(None, description="List of BrandAttributes")
    
# completion = client.beta.chat.completions.parse(
#     model="gpt-4o-2024-08-06",
#     messages=[
#         {"role": "system", "content": "Be a helpful assistant."},
#         {"role": "system", "content": "List 100 best selling American clothing brands and, for each brand, list which one of the emotions in the given list are most closely associated with."},
#         {"role": "user", "content": "List 100 best selling American clothing brands and, for each brand, list which one of the emotions in the given list are most closely associated with."}
#     ],
#     response_format=BrandChars,
# )

# out = json.loads(completion.choices[0].message.content)
# brand_emotions = out
# out = list(out.values())[0]
# df_brand = pd.DataFrame(data = [i.values() for i in out], columns = ['Name','characteristic'])
# df_brand

# #Take the example song, and list which one of the emotions in the given list are most closely associated with the song.")

# client = OpenAI(api_key=k)

# Characteristic = Enum('Characteristic', dict([(i, i) for i in emotions])) 

# class BookAttributes(BaseModel):
#     book_name: str
#     book_writer :str = Field(None, description="writer as str")
#     characteristics: List[Characteristic]

# class BooksChars(BaseModel):
#     a: List[BookAttributes] = Field(None, description="List of book attributes.")

# prompt = """
# As prompt, take the book 'Summer Island', list its writer, and a list of the emotions from the given list that are most closely associated with the book.
# """

# completion = client.beta.chat.completions.parse(
#     model="gpt-4o-2024-08-06",
#     messages=[
#         {"role": "system", "content": "Be a helpful assistant."},
#         {"role": "user", "content": prompt}
#     ],
#     response_format=BooksChars,
# )
# out = json.loads(completion.choices[0].message.content)
# out = list(out.values())[0]
# df_book = pd.DataFrame(data = [i.values() for i in out], columns = ['Name','Writer', 'characteristic'])
# df_book

# #MultilabelBinarize brands and song
# from sklearn.preprocessing import MultiLabelBinarizer

# mlb = MultiLabelBinarizer(classes=emotions)

# df_brand2 = pd.DataFrame(mlb.fit_transform(df_brand['characteristic']), columns = emotions)
# df_brand_mb = pd.merge(df_brand2, df_brand[['Name']], how ='left', left_index=True, right_index=True)
# df_brand_mb.head()

# mlb = MultiLabelBinarizer(classes=emotions)

# df_book2 = pd.DataFrame(mlb.fit_transform(df_book['characteristic']), columns = emotions)
# df_book_mb = pd.merge(df_book2, df_book[['Name']], how ='left', left_index=True, right_index=True)
# df_book_mb.head()

# df_b_T = df_brand_mb.set_index('Name').T
# df_bo_T = df_book_mb.set_index('Name').T

# df_bo_T


# df_b_T.corrwith(df_bo_T, axis = 1)


#---------------
#2nd method, I tried, was to go through every brand from 10, and every emotion from 50, for every pair assign an association value (0 to 500):

# # Prompt GPT to determine the association between brand and emotion pairs based on semantic or contextual understanding.
  
# # Define the Pydantic models for the input and output
# class ObjectPair(BaseModel):
#     object1: str
#     object2: str

# class RelationRequest(BaseModel):
#     list1: List[str]  # First list of objects
#     list2: List[str]  # Second list of objects
#     model: str = Field(default="gpt-4o-2024-08-06")  # OpenAI model version
#     min_value: float = 0  # Minimum value of the normalized range
#     max_value: float = 500  # Maximum value of the normalized range 

# class RelationResponse(BaseModel):
#     pair: ObjectPair
#     association_score: float  # The score provided by GPT

# class OpenAIRelationQuantifier:
#     @staticmethod
#     def _generate_prompt(pair: ObjectPair, min_value: float, max_value: float) -> str:
#         """Generate the prompt for GPT to evaluate the association between two objects."""
#         return f"On a scale from {min_value} to {max_value}, how strongly are the following two items related?\n\n" \
#                f"Item 1: {pair.object1}\nItem 2: {pair.object2}\n" \
#                f"Please provide a score and a short explanation of their relationship."

#     @staticmethod
#     def quantify_relations(request: RelationRequest) -> List[RelationResponse]:
#         """Quantify the relationships between each pair of objects using GPT."""
#         client = OpenAI(api_key=k)  

#         results = []
#         for object1 in request.list1:
#             for object2 in request.list2:
#                 pair = ObjectPair(object1=object1, object2=object2)
#                 prompt = OpenAIRelationQuantifier._generate_prompt(pair, request.min_value, request.max_value)

#                 # Send the prompt to GPT using beta.chat.completions.parse method
#                 response = client.beta.chat.completions.parse(
#                     model=request.model,
#                     messages=[
#                         {"role": "system", "content": "You are an expert at analyzing relationships between concepts."},
#                         {"role": "user", "content": prompt}
#                     ]
#                 )

#                 # Extract GPT's parsed response
#                 gpt_reply = response.choices[0].message.content
#                 score_line = gpt_reply.splitlines()[0]
#                 score = float(score_line.split()[-1].rstrip('.'))  # Assumes the score is at the end of the first line

#                 # Append the result
#                 results.append(RelationResponse(pair=pair, association_score=score))

#         return results

# # Usage

# # Define two lists of objects to compare
# list1 = brands[:3]
# list2 = emotions[:5]

# # Create the relation request object
# relation_request = RelationRequest(list1=list1, list2=list2, min_value=0, max_value=500)

# # Quantify the relationships between objects in the two lists
# relations = OpenAIRelationQuantifier.quantify_relations(relation_request)

# # Display the results
# for relation in relations:
#     print(f"Pair: ({relation.pair.object1}, {relation.pair.object2}) - Association Score: {relation.association_score:.2f}")
# relations


# df=pd.DataFrame()
# df['brand']= [i.pair.object1 for i in relations]
# df['emotion']= [i.pair.object2 for i in relations]
# df['score']= [i.association_score for i in relations]

# print(df)
# df = df.pivot(index='brand',columns ='emotion', values='score')
